In [1]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__

'1.1.0'

In [6]:
data=np.loadtxt("german.data-numeric")

In [8]:
n,l=data.shape
for j in range(l-1):
    meanVal=np.mean(data[:,j])
    stdVal=np.std(data[:,j])
    data[:,j]=(data[:,j]-meanVal)/stdVal